# Qubit Characterization: HDAWG + UHFQA

Here we show some basic examples for how to use the *HDAWG* and *UHFQA* together in an experiment. To ensure that the two devices use the same Data Server, we use the *zhinst-toolkit's* `MultiDeviceConnection` and to setup and connect a Data Server and then connect the two devices to it.

In [1]:
import zhinst.toolkit as tk
import numpy as np

mdc = tk.MultiDeviceConnection(host="10.42.0.226")
mdc.setup()

Successfully connected to data server at 10.42.0.2268004 api version: 6


In [2]:
mdc.connect_device(tk.HDAWG("hdawg 1", "dev8030"))
mdc.connect_device(tk.UHFQA("uhfqa 1", "dev2266"))

Successfully connected to device DEV8030 on interface 1GBE
Successfully connected to device DEV2266 on interface 1GBE


References to the instrument drivers are held in attributes of the `MultiDeviceConnection`.

In [3]:
hdawg = mdc.hdawgs["hdawg 1"]
uhfqa = mdc.uhfqas["uhfqa 1"]

## Aligned Waveform Playback

The *'Simple'* sequence type allows the user to upload their own numpy arrays for as waveforms. The values in the array thereby correspond to the samples in the waveform. As a straightforward first example we use one *AWG Core* of the *HDAWG* to play a rectangular wave and to trigger the playback on the *UHFQA*. The *'Mark'* output of the AWG on the *HDAWG* needs to be connected with a coax cable to the *Trig/Ref 1* on the *UHFQA*.


```
              HDAWG 1                                                         t=0
          +-----------+                        :______________________________:         :
     +----+   AWG 1   |  Trigger              _|                         XXXXX|_________:_
     |    +-----------+  ("Send Trigger")      :                              :         :
     |                                         :                              :         :
     |                                         :                              :         :
     |     UHFQA                               :                              :         :
     |    +-----------+                        :                              :         :
     +----->          | Readout               _:______________________________|XXXXX|___:_
          +-----------+ ("External Trigger")
```

We configure both AWGs with the same number or `repetitions` and the same `period`. The *trigger mode* of the AWG Core on the HDAWG is set to `"Send Trigger"`, i.e. to send out a trigger signal at the start of every period. The *trigger mode* of the *UHFQA* is set to wait for a trigger input at the start of every period with `"External Trigger"`. 

In [4]:
# configure sequence parameters of master trigger
hdawg.awgs[0].set_sequence_params(
    sequence_type="Simple",
    period=20e-6,
    repetitions=1000,
    trigger_mode="Send Trigger",     # send out the trigger signal 
    alignment="End with Trigger",    # end waveform on t=0 
)

# configure sequence parameters of UHFQA
uhfqa.awg.set_sequence_params(
    sequence_type="Simple",
    period=20e-6,
    repetitions=1000,
    trigger_mode="External Trigger", # wait for the trigger signal 
    alignment="Start with Trigger",  # start waveform on t=0
)

By setting the parameter `alignment` to `"End with Trigger"` on the HDAWG and `"Start with Trigger"` on the UHFQA, we make sure that the waveforms from different devices are played right after each other.

For both AWG Cores we add a waveform to the queue. The waveform is defined by a numpy array (`np.ones(...)`) of certain length. With the method `compile_and_upload_waveforms()` we tell the AWG to compile the corresponding sequence program and upload the waveforms in the queue.

In [5]:
# queue rectangular waveform on HDAWG channel 1
hdawg.awgs[0].reset_queue()
hdawg.awgs[0].queue_waveform(np.ones(1000), -np.ones(1000)) 
hdawg.awgs[0].compile_and_upload_waveforms()

# queue rectangular waveform on UHFQA channel 2
uhfqa.awg.reset_queue()
uhfqa.awg.queue_waveform(-np.ones(1000), np.ones(1000)) 
uhfqa.awg.compile_and_upload_waveforms()

Current length of queue: 1
Compilation successful
hdawg 1-0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.023009 s
Current length of queue: 1
Compilation successful
uhfqa 1-0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.067884 s


Run the experiment! First, make sure the outputs are on. Start the AWG of the *UHFQA* first, then the Master AWG on the *HDAWG*. 

In [6]:
# turn outputs on
hdawg.awgs[0].outputs(("on", "on"))
uhfqa.awg.outputs(("on", "on"))

# start uhfqa awg, waiting for trigger
uhfqa.awg.run()

# start master awg
hdawg.awgs[0].run()
hdawg.awgs[0].wait_done()

Verify the outputs with a scope, e.g. the built-in scope of the *UHFQA*. The rectangular waveform output of the *HDAWG* should be followed directly by the waveform from the *UHFQA*. 

## Resonator Spectroscopy

For a resonator spectroscopy we can use the predefined `"Pulsed Spectroscopy"` sequence on the UHFQA. It plays a rectangular waveform of duration `pulse_length`. We configure one AWG Core of the *HDAWG* to a `"Trigger"` sequence to trigger the spectroscopy pulses on the *UHFQA* (`trigger_mode="External Trigger"`). 

In [7]:
# assign AWG Cores
trigger = hdawg.awgs[0]
readout = uhfqa.awg

# configure trigger AWG
trigger.set_sequence_params(
    sequence_type="Trigger",
    period=10e-6,
    repetitions=1e3,
)
trigger.compile()

# configure UHFQA to Pulsed Spectroscopy
readout.set_sequence_params(
    sequence_type="Pulsed Spectroscopy",
    period=10e-6,
    pulse_lenth=2e-6,
    repetitions=1e3,
    trigger_mode= "External Trigger",   # UHFQA triggered by Master Trigger
)
readout.compile()

Compilation successful
hdawg 1-0: Sequencer status: ELF file uploaded
Compilation successful
uhfqa 1-0: Sequencer status: ELF file uploaded


The spectroscopy sequence enables output modulation on the *UHFQA*. The signals on the two output channels are modulated with the sine and cosine of the internal oscillator. Its frequency is set by the parameter `uhfqa.nodetree.osc.freq`. On the signal acquisition side, the *integration mode: "Spectroscopy"* is used, which demodulates the signal input with the same internal oscillator.

A sweep of the IF modulation frequency could be done like this: 

In [ ]:
# configure results to length 1000 and 1 hardware averages
uhfqa.result_source("Integration")
uhfqa.nodetree.qa.result.length(1e3)
uhfqa.nodetree.qa.result.averages(1)
modulation_freq = uhfqa.nodetree.osc.freq

# define the sweep points and an empty array for results
frequencies = np.linspace(50e6, 100e6, 101)
results = np.array([])

for f in frequencies:
    modulation_freq(f)                               # set modulation frequency
    readout.run()                                    # start readout AWG
    trigger.run()                                    # start trigger AWG
    trigger.wait_done()                              # wait until trigger AWG has finished
    avg_result = np.mean(uhfqa.channels[0].result()) # average the result vector
    results = np.append(results, avg_result)         # append to results

## Qubit Spectroscopy

A drive pulse for qubit spectroscopy can be implemented in one of two ways. Either the predefined `"Rabi"` sequence (see below) is used with a single amplitude point and a long pulse width. The option would be to use the `"Simple"` sequence and upload the drive pulse of choice as a *numpy array*. For simplicity we will show the latter option and play a simle rectangular pulse of *5 us* duration on the *HDAWG*.

On the *UHFQA*, we use the predefined `"Readout"` sequence. This sequence creates a readout tone for every enabled *Readout Channel* of the *UHFQA* at their respective `readout_frequency`. By enabling the readout channel we also program the integration weights of the readout channel to demodulate the signal at the given `readout_frequency`. 

In [8]:
# assign AWGs
drive = hdawg.awgs[0]
readout = uhfqa.awg
channel = uhfqa.channels[0]

# configure trigger AWG
drive.set_sequence_params(
    sequence_type="Simple",
    period=10e-6,
    repetitions=1e3,
)
drive.reset_queue()
wave = np.ones(int(2.4e9 * 5e-6))  # sampling rate x pulse duration
drive.queue_waveform(wave, wave)
drive.compile_and_upload_waveforms()

# configure UHFQA to Readout
channel.enable()
channel.readout_frequency(123e6)
readout.set_sequence_params(
    sequence_type="Readout",
    period=10e-6,
    pulse_lenth=2e-6,
    repetitions=1e3,
    trigger_mode= "External Trigger",   # UHFQA triggered by Master Trigger
)
readout.compile()

Current length of queue: 1
Compilation successful
hdawg 1-0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.071 s
Compilation successful
uhfqa 1-0: Sequencer status: ELF file uploaded


To sweep the frequency of the drive signal, we want to enable modulation of the ouput signal of the qubit drive at with the internal oscillator. This is done by enabling the IQ Modulation mode of the AWG Core (modulation with the internal oscillator for IQ signal generation). The frequency of the oscillator can then be swept over the desired range.

In [ ]:
frequencies = np.linspace(20e6, 50e6, 101)
results = np.array([])

# enable IQ Modulation mode
drive.enable_iq_modulation()

for f in frequencies:
    drive.modulation_freq(f)
    readout.run()
    drive.run()
    drive.wait_done()
    avg_result = np.mean(channel.result())
    results = np.append(results, avg_result)


## Rabi Sequence

The *Rabi Sequence* plays a simple Gaussian pulse that is defined by the sequence parameters `pulse_width` and `pulse_truncation`. The parameter `pulse_amplitudes` expects a *list* or *array* of values for the amplitude sweep. With the IQ Modulation mode enabled a typical Rabi sequence would look like this:

In [17]:
# assign AWGs
drive = hdawg.awgs[0]
readout = uhfqa.awg
channel = uhfqa.channels[0]

# define numpy array with Rabi Amplutdes
rabi_amplitudes = np.linspace(0, 1.0, 101)

# configure Rabi Amplitude Sweep
hdawg.awgs[0].enable_iq_modulation()
hdawg.awgs[0].modulation_freq(100e6)

hdawg.awgs[0].set_sequence_params(
    sequence_type="Rabi",
    pulse_width=50e-9,
    pulse_amplitudes=rabi_amplitudes,
)
hdawg.awgs[0].compile()

# configure UHFQA to Readout
uhfqa.nodetree.qa.result.length(1e3 * len(rabi_amplitudes))
uhfqa.nodetree.qa.result.averages(1)

channel.enable()
channel.readout_frequency(123e6)
readout.set_sequence_params(
    sequence_type="Readout",
    period=10e-6,
    pulse_lenth=2e-6,
    repetitions=1e3 * len(rabi_amplitudes),  # play averages x sweep-points readout pulses
    trigger_mode= "External Trigger",        # UHFQA triggered by Master Trigger
)
readout.compile()

Compilation successful
hdawg 1-0: Sequencer status: ELF file uploaded
Compilation successful
uhfqa 1-0: Sequencer status: ELF file uploaded


In [ ]:
readout.run()
drive.run()
drive.wait_done()
result = channel.result()

## Time Domain Characterization: T1 & T2*

Time domain characterization can be done with predefined sequences *T1* and *T2 Ramsey*. They work in the same way as the *Rabi Sequence*. However, instead of passing an array of sweep values to the parameter `pulse_amplitudes`, the parameter is called `time_delays`. This parameter defines the  

In [17]:
# assign AWGs
drive = hdawg.awgs[0]
readout = uhfqa.awg
channel = uhfqa.channels[0]

# define numpy array with Delay Times
delay_times = np.linspace(0.5e-6, 20e-6, 40)

# configure Rabi Amplitude Sweep
hdawg.awgs[0].enable_iq_modulation()
hdawg.awgs[0].modulation_freq(100e6)

hdawg.awgs[0].set_sequence_params(
    sequence_type="T1",
    pulse_width=50e-9,
    delay_times=delay_times,
)
hdawg.awgs[0].compile()

# configure UHFQA to Readout
uhfqa.nodetree.qa.result.length(1e3 * len(delay_times))
uhfqa.nodetree.qa.result.averages(1)

channel.enable()
channel.readout_frequency(123e6)
readout.set_sequence_params(
    sequence_type="Readout",
    period=10e-6,
    pulse_lenth=2e-6,
    repetitions=1e3 * len(delay_times),  # play averages x sweep-points readout pulses
    trigger_mode= "External Trigger",        # UHFQA triggered by Master Trigger
)
readout.compile()

Compilation successful
hdawg 1-0: Sequencer status: ELF file uploaded
Compilation successful
uhfqa 1-0: Sequencer status: ELF file uploaded


In [ ]:
readout.run()
drive.run()
drive.wait_done()
result = channel.result()

## Multi-Qubit Characterization: Master Trigger Waveform Playback

```
              HDAWG 1
           +-----------+                    ______________________________
+----<----+   AWG 1   |  Trigger          _|                              |_________:_
|         +-----------+                    :                              :         :
+----+----->  AWG 2   |  AWGs[0]          _:________________________|XXXXX|_________:_
     |    |-----------+                    :                              :         :
     +----->  AWG 3   |  AWGs[1]          _:_____________________|XXXXXXXX|_________:_
     |    |-----------+                    :                              :         :
     +----->  AWG 4   |  AWGs[2]          _:__________________|XXXXXXXXXXX|_________:_
     |    +-----------+                    :                              :         :
     |                                     :                              :         :
     |     UHFQA                           :                              :         :
     |    +-----------+                    :                              :         :
     +----->          | Readout           _:______________________________|XXXXX|___:_
          +-----------+
```

In [4]:
# group and rename AWG Cores
trigger = hdawg.awgs[0]
awgs = hdawg.awgs[1:]
readout = uhfqa.awg

# common sequence parameters
period = 20e-6
repetitions = 1000

# configure trigger AWG
trigger.set_sequence_params(
    sequence_type="Trigger",
    period=period,
    repetitions=repetitions,
)
trigger.compile()

# configure triggered AWG Cores
for awg in awgs:
    awg.set_sequence_params(
        sequence_type="Simple",
        period=period,
        repetitions=repetitions,
        alignment="End with Trigger",
        trigger_mode="External Trigger",
    )
readout.set_sequence_params(
    sequence_type="Simple",
    period=period,
    repetitions=repetitions,
    alignment="Start with Trigger",
    trigger_mode="External Trigger",
)   

Compilation successful
hdawg 1-0: Sequencer status: ELF file uploaded


In [5]:
# reset queues
[awg.reset_queue() for awg in awgs]
readout.reset_queue()

# queue waveforms
awgs[0].queue_waveform(np.ones(800), -np.ones(800))
awgs[1].queue_waveform(np.ones(1000), -np.ones(1000))
awgs[2].queue_waveform(np.ones(1200), -np.ones(1200))
readout.queue_waveform(np.ones(800), -np.ones(800))

# compile and upload triggered AWGs 
[awg.compile_and_upload_waveforms() for awg in awgs]
readout.compile_and_upload_waveforms()

Current length of queue: 1
Current length of queue: 1
Current length of queue: 1
Current length of queue: 1
Compilation successful
hdawg 1-1: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.06787 s
Compilation successful
hdawg 1-2: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.017991 s
Compilation successful
hdawg 1-3: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.022224 s
Compilation successful
uhfqa 1-0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.063616 s
